In [ ]:
## 1. Mount Google Drive and Install Dependencies
from google.colab import drive
drive.mount('/content/drive')

In [36]:
!pip install -q segmentation-models albumentations

In [37]:
## 2. Import Libraries
import os
import numpy as np
import tensorflow as tf
import albumentations as A
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence
import cv2
import matplotlib.pyplot as plt

In [38]:
## 3. Define Custom Metrics (Load from Week 2)
from tensorflow.keras import backend as K

def iou_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [39]:
## 4. Load Week 2 Model from Drive
model = load_model('/content/drive/MyDrive/disaster_model_week2.h5',
                   custom_objects={'iou_coef': iou_coef, 'dice_coef': dice_coef})

In [40]:
## 5. Define Albumentations-Based Augmentations
augmentations = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=20, p=0.5),
])

In [41]:
## 6. Create a Custom Data Generator with Augmentations
class AugmentedDataGen(Sequence):
    def __init__(self, image_paths, mask_paths, batch_size, augmentations, img_size=(256, 256)):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.batch_size = batch_size
        self.augment = augmentations
        self.img_size = img_size

    def __len__(self):
        return len(self.image_paths) // self.batch_size

    def __getitem__(self, idx):
        batch_images = self.image_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_masks = self.mask_paths[idx * self.batch_size:(idx + 1) * self.batch_size]

        images = []
        masks = []

        for img_path, mask_path in zip(batch_images, batch_masks):
            image = cv2.imread(img_path)
            image = cv2.resize(image, self.img_size)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, self.img_size)

            if self.augment:
                augmented = self.augment(image=image, mask=mask)
                image = augmented['image']
                mask = augmented['mask']

            image = image / 255.0
            mask = (mask > 127).astype(np.float32)
            mask = np.expand_dims(mask, axis=-1)

            images.append(image)
            masks.append(mask)

        return np.array(images), np.array(masks)

In [42]:
## 7. Prepare Training and Validation Generators

from glob import glob
train_images = sorted(glob('/content/drive/MyDrive/flood-dataset/train/Image/*.jpg'))
train_masks = sorted(glob('/content/drive/MyDrive/flood-dataset/train/Mask/*.png'))
val_images = sorted(glob('/content/drive/MyDrive/flood-dataset/val/Image/*.jpg'))
val_masks = sorted(glob('/content/drive/MyDrive/flood-dataset/val/Mask/*.png'))


batch_size = 8
train_gen = AugmentedDataGen(train_images, train_masks, batch_size, augmentations)
val_gen = AugmentedDataGen(val_images, val_masks, batch_size, None)

In [43]:
## 8. Fine-Tune the Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=[iou_coef, dice_coef])

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

KeyboardInterrupt: 

In [ ]:
## 9. Save the Fine-Tuned Model
model.save('/content/drive/MyDrive/disaster_model_week3.h5')
print("✅ Week 3 fine-tuned model saved.")

In [44]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

# Redefine custom metrics to be used during model loading
def iou_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Load the final model from Google Drive
model = load_model('/content/drive/MyDrive/disaster_model_week3.h5',
                   custom_objects={'iou_coef': iou_coef, 'dice_coef': dice_coef})

print("✅ Final Week 4 model loaded successfully.")


✅ Final Week 4 model loaded successfully.


In [ ]:
# Week 4: Evaluation, Reporting, and Finalization

## 10. Evaluate Model on Validation Set
results = model.evaluate(val_gen)
print("Validation Loss:", results[0])
print("Validation IoU:", results[1])
print("Validation Dice Coefficient:", results[2])

In [ ]:
## 11. Visualize Predictions
def show_predictions(generator, model, num=3):
    import random
    batch = next(iter(generator))
    images, masks = batch

    for i in range(num):
        idx = random.randint(0, len(images) - 1)
        image = images[idx]
        true_mask = masks[idx]
        pred_mask = model.predict(np.expand_dims(image, 0))[0] > 0.5

        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(image)
        plt.title('Image')

        plt.subplot(1, 3, 2)
        plt.imshow(true_mask[..., 0], cmap='gray')
        plt.title('True Mask')

        plt.subplot(1, 3, 3)
        plt.imshow(pred_mask[..., 0], cmap='gray')
        plt.title('Predicted Mask')

        plt.show()

show_predictions(val_gen, model)

In [ ]:
## 12. Save Final Model and Metrics Report
model.save('/content/drive/MyDrive/disaster_model_week4_final.h5')
print("✅ Final model saved for Week 4.")

In [45]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

# Redefine custom metrics to be used during model loading
def iou_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Load the final model from Google Drive
model = load_model('/content/drive/MyDrive/disaster_model_week4_final.h5',
                   custom_objects={'iou_coef': iou_coef, 'dice_coef': dice_coef})

print("✅ Final Week 4 model loaded successfully.")


✅ Final Week 4 model loaded successfully.


In [46]:
with open('/content/drive/MyDrive/disaster_model_week4_metrics.txt', 'w') as f:
    f.write(f"Validation Loss: {results[0]:.4f}\n")
    f.write(f"Validation IoU: {results[1]:.4f}\n")
    f.write(f"Validation Dice: {results[2]:.4f}\n")
print("📊 Metrics saved to Google Drive.")

📊 Metrics saved to Google Drive.


In [47]:
# Front end Streamlit code
!pip install -q streamlit pyngrok

In [48]:
%%writefile app.py
import streamlit as st
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from PIL import Image

# Load model and metrics
@st.cache_resource
def load_segmentation_model():
    from tensorflow.keras import backend as K

    def iou_coef(y_true, y_pred, smooth=1):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)

    def dice_coef(y_true, y_pred, smooth=1):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

    model = load_model('/content/drive/MyDrive/disaster_model_week4_final.h5',
                       custom_objects={'iou_coef': iou_coef, 'dice_coef': dice_coef})
    return model

model = load_segmentation_model()

st.title("🏚️ Disaster Image Segmentation")
st.write("Upload a satellite image to predict flood-damaged regions.")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file).convert("RGB")
    image = image.resize((256, 256))
    img_array = np.array(image) / 255.0

    st.image(image, caption='Uploaded Image', use_container_width=True)

    pred_mask = model.predict(np.expand_dims(img_array, axis=0))[0] > 0.5

    st.subheader("Predicted Mask")
    from PIL import Image

    mask_image = Image.fromarray((pred_mask.squeeze() * 255).astype(np.uint8))
    st.image(mask_image, caption="Predicted Mask", use_container_width=True)



Overwriting app.py


In [49]:
# 1. Kill any previous streamlit processes (clean state)
!pkill -f streamlit

# 2. Start Streamlit in background (from your script file)
!nohup streamlit run app.py --server.port 8501 > /dev/null 2>&1 &

# 3. Start ngrok correctly with string port
from pyngrok import ngrok

# Optional: set auth token if needed (only once per account/session)
ngrok.set_auth_token("30OywoUFWvhuBkGYJoOdAW8oSgj_2ECDW7k24BqJQvvmrjRJb")

# 4. Connect to the port using correct parameter
public_url = ngrok.connect("8501", bind_tls=True)

print("✅ Streamlit is live at:", public_url)


✅ Streamlit is live at: NgrokTunnel: "https://f14cfe1f44b0.ngrok-free.app" -> "http://localhost:8501"
